----
FOURTH DATASET

In [1]:
import os
from PIL import Image
import pandas as pd

In [2]:
def find_directories_with_name(root_folder, target_name):
    matching_directories = []

    for dirpath, dirnames, _ in os.walk(root_folder):
        for dirname in dirnames:
            if dirname == target_name:
                matching_directories.append(os.path.join(dirpath, dirname))

    return matching_directories

In [3]:
root_folder = "/Users/osiprovin/Desktop/ml:dl/CV/Parking detection/Data/FOURTH DATASET/CNR-EXT_FULL_IMAGE_1000x750 3"
cameras_paths = {}
bounding_boxes_by_cam = {}

for camera_num in range(1, 10):
    camera_name = f'camera{camera_num}'
    cameras_paths[camera_name] = find_directories_with_name(root_folder, camera_name)

    bounding_boxes_path = os.path.join(root_folder, f'{camera_name}.csv')
    bounding_boxes_by_cam[camera_name] = pd.read_csv(bounding_boxes_path)

bnbs was created for
last size 2592x1944
new 1000x750
lets resize bnbs

In [4]:
def resize_bnb(bounding_box):
    scale_x = 1000 / 2592
    scale_y = 750 / 1944
    idx, x, y, w, h = bounding_box
    x *= scale_x
    y *= scale_y
    w *= scale_x
    h *= scale_y
    return idx, x, y, w, h

In [5]:
def prepare_bounding_box(bounding_box):
    idx, x, y, w, h = resize_bnb(bounding_box)
    left, upper = x, y
    right, lower = x + w, y + h
    prepared_bounding_box = left, upper, right, lower
    return idx, prepared_bounding_box

format of is pklot occupied data
RAINY/2016-02-12/camera7/R_2016-02-12_12.25_C07_275.jpg 1
lets save our pictures with this type of path.

In [6]:
def get_save_path(image_path, bnb_idx, occupieds):
    splitted_path = image_path.split("/")
    DATA_path = ""
    for idx, folder_name in enumerate(splitted_path):
        if folder_name == "FULL_IMAGE_1000x750":
            whether = splitted_path[idx+1]
            date = splitted_path[idx+2]
            camera_name = splitted_path[idx+3]
            time = splitted_path[idx+4][11:15]

            hours = time[0:2]
            minutes = time[2:4]
            camera_num = camera_name[-1]

            DATA_path = "/".join(splitted_path[0:idx - 1])
            break

    image_name = f'{whether[0]}_{date}_{hours}.{minutes}_C0{camera_num}_{bnb_idx}.jpg'
    workspace_path = os.path.join(whether, date, camera_name, image_name)
    
    try:
        busy_free = "busy" if int(occupieds[workspace_path]) else "free"
        full_image_path = os.path.join(DATA_path, "CROPPED", busy_free, workspace_path)
        return full_image_path
    
    except KeyError:
        print(f"Image: {workspace_path} is not annotated :(")
        return None

In [7]:
def save_image(image, image_path):
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    image.save(image_path)

In [8]:
def crop_image_by_bnbs(image_path, bounding_boxes, occupieds):
    image = Image.open(image_path)
    for idx, bounding_box in bounding_boxes.iterrows():
        bnb_idx, prepared_bounding_box = prepare_bounding_box(bounding_box)
        cropped_image = image.crop(prepared_bounding_box)
        save_path = get_save_path(image_path, bnb_idx, occupieds)
        if save_path is not None:
            save_image(cropped_image, save_path)

In [9]:
def crop_images(camera_paths, camera_bounding_boxes, occupieds):
    for camera_path in camera_paths:
        if os.path.isdir(camera_path):
            for image_name in os.listdir(camera_path):
                if image_name.endswith(".jpg"):
                    image_path = os.path.join(camera_path, image_name)
                    crop_image_by_bnbs(image_path, camera_bounding_boxes, occupieds)

In [10]:
txt_occupied_all_path = "/Users/osiprovin/Desktop/ml:dl/CV/Parking detection/Data/FOURTH DATASET/splits/CNRPark-EXT/all.txt"
occupieds = {}
for line in open(txt_occupied_all_path).readlines():
    image_name, is_occupied = line.strip().split()
    occupieds[image_name] = is_occupied

In [11]:
for camera_name in cameras_paths.keys():
    crop_images(cameras_paths[camera_name], bounding_boxes_by_cam[camera_name], occupieds)

Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_603.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_604.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_605.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_606.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_607.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_608.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_184.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_185.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_186.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_187.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_C01_188.jpg is not annotated :(
Image: OVERCAST/2015-11-16/camera1/O_2015-11-16_17.10_

----
FIFTH DATASET

In [12]:
import xml.etree.ElementTree as ET

In [13]:
images_folder_path = "/Users/osiprovin/Desktop/ml:dl/CV/Parking detection/Data/FIFTH DATASET/CAR PARKING LOT DATASET/images"
boxes_folder_path = "/Users/osiprovin/Desktop/ml:dl/CV/Parking detection/Data/FIFTH DATASET/CAR PARKING LOT DATASET/boxes"

In [14]:
def extract_data(xml_path):
    coordinates = []

    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.findall('object'):
        label = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        ymin = bndbox.find('ymin').text
        xmax = bndbox.find('xmax').text
        ymax = bndbox.find('ymax').text

        coordinates.append((label, (int(xmin), int(ymin), int(xmax), int(ymax))))

    return coordinates

In [15]:
def get_rotated_duplicates(image, angles):
    rotated_images = []
    for angle in angles:
        rotated_image = image.rotate(angle, expand=True)
        rotated_images.append(rotated_image)
    return rotated_images

In [16]:
def create_subfolder(folder_path, subfolder_name):
    subfolder_path = os.path.join(folder_path, subfolder_name)
    os.makedirs(subfolder_path, exist_ok=True)
    return subfolder_path

In [17]:
path = "/Users/osiprovin/Desktop/ml:dl/CV/Parking detection/Data/FIFTH DATASET/CROPPED"

busy = "busy"
free = "free"

busy_path = create_subfolder(path, busy)
free_path = create_subfolder(path, free)

In [18]:
cropped_images = []

save_idx = 0

for filename in os.listdir(boxes_folder_path):
    imagename = f"{os.path.splitext(filename)[0]}.png"

    if filename.endswith(".xml"):
        xml_path = os.path.join(boxes_folder_path, filename)
        labels_boxes = extract_data(xml_path)

        image_path = os.path.join(images_folder_path, imagename)
        image = Image.open(image_path)

        for label, box in labels_boxes:
            cropped_image = image.crop(box)
            cropped_rotated_images = get_rotated_duplicates(cropped_image, [0, 90, 180, 270])

            save_folder_path = busy_path if label == "busy" else free_path
            
            for cropped_rotated_image in cropped_rotated_images:
                cropped_rotated_image_name = f"{save_idx}.png"
                save_path = os.path.join(save_folder_path, cropped_rotated_image_name)
                cropped_rotated_image.save(save_path)
                save_idx += 1